In [ ]:
import sys
import numpy as np
import cv2

# Patch numpy float for ByteTrack compatibility
np.float = float

# Add ByteTrack repo to sys.path
sys.path.append('./ByteTrack')
from yolox.tracker.byte_tracker import BYTETracker


In [ ]:
!git clone https://github.com/ifzhang/ByteTrack.git

In [ ]:
%cd ByteTrack
!pip install -r requirements.txt
!python setup.py develop

In [ ]:
%cd ..
import sys
sys.path.append('/content/ByteTrack')

In [ ]:
import numpy as np
np.float = float

In [ ]:
from ultralytics import YOLO

# Load YOLOv8 model (update path as needed)
model = YOLO('best.pt')

# Set ByteTrack tracker parameters
class Args:
    track_thresh = 0.5
    track_buffer = 30
    match_thresh = 0.8
    min_box_area = 10
    mot20 = False

tracker = BYTETracker(Args())


In [ ]:
# Set input video path
video_path = 'traffic.mp4'

# Open video
cap = cv2.VideoCapture(video_path)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output_tracked.mp4', fourcc, fps, (width, height))

# Get class name mapping from YOLO model
class_names = model.names if isinstance(model.names, dict) else dict(enumerate(model.names))

# Prepare results file
result_file = open('results.txt', 'w')

# IDs of classes to track, according to your data.yaml ['others', 'car', 'van', 'bus']
follow_classes = [0, 1, 2, 3] 

In [ ]:
frame_id = 0
id_offset = None

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    frame_id += 1

    # Run YOLO detection on the frame
    results = model.predict(source=frame, verbose=False)
    boxes = results[0].boxes.xyxy.cpu().numpy()
    scores = results[0].boxes.conf.cpu().numpy()
    classes = results[0].boxes.cls.cpu().numpy()

    # Filter only selected vehicle classes
    vehicle_indices = np.where(np.isin(classes, follow_classes))[0]
    dets = []
    det_classes = []
    for idx in vehicle_indices:
        x1, y1, x2, y2 = boxes[idx]
        score = scores[idx]
        dets.append([x1, y1, x2, y2, score])
        det_classes.append(int(classes[idx]))
    dets = np.array(dets)
    if len(dets) == 0:
        dets = np.empty((0, 5))

    img_info = [frame.shape[0], frame.shape[1]]
    img_size = (frame.shape[0], frame.shape[1])

    # Track objects using ByteTrack
    online_targets = tracker.update(dets, img_info, img_size)

    # Set ID offset for shifting IDs to start from 1
    if id_offset is None:
        if len(online_targets) > 0:
            id_offset = min([t.track_id for t in online_targets])
        else:
            id_offset = 0

    active_ids = []
    # Draw boxes, labels, and save to txt
    for t in online_targets:
        tlwh = t.tlwh
        track_id = t.track_id
        shifted_id = track_id - id_offset + 1  # Shift IDs to start from 1
        active_ids.append(shifted_id)

        # Find class name by matching detection to track with IoU
        t_box = np.array([tlwh[0], tlwh[1], tlwh[0]+tlwh[2], tlwh[1]+tlwh[3]])
        best_iou = 0
        best_cls = -1
        for det_box, cls in zip(dets, det_classes):
            xx1 = max(t_box[0], det_box[0])
            yy1 = max(t_box[1], det_box[1])
            xx2 = min(t_box[2], det_box[2])
            yy2 = min(t_box[3], det_box[3])
            w = max(0., xx2-xx1)
            h = max(0., yy2-yy1)
            intersection = w * h
            area1 = (t_box[2]-t_box[0]) * (t_box[3]-t_box[1])
            area2 = (det_box[2]-det_box[0]) * (det_box[3]-det_box[1])
            union = area1 + area2 - intersection
            iou = intersection / union if union > 0 else 0
            if iou > best_iou:
                best_iou = iou
                best_cls = cls

        class_name = class_names.get(best_cls, "unknown")
        x1, y1, w, h = [int(i) for i in tlwh]
        # Draw bounding box and label on the frame
        cv2.rectangle(frame, (x1, y1), (x1 + w, y1 + h), (0,255,0), 2)
        label = f"{shifted_id}: {class_name}"
        cv2.putText(frame, label, (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,255,0), 2)
        # Write results to text file (frame_id, class_name, x1, y1, w, h)
        result_file.write(f"{frame_id},{class_name},{x1},{y1},{w},{h}\n")

    # Calculate traffic density level based on active vehicle count
    active_vehicle_count = len(active_ids)
    if active_vehicle_count <= 5:
        density_level = "Low"
    elif active_vehicle_count <= 10:
        density_level = "Medium"
    else:
        density_level = "High"

    # Write traffic density text at the bottom of the frame
    cv2.putText(frame, f"Traffic: {density_level}", (50, height - 50),
                cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 6)

    # Write the processed frame to output video
    out.write(frame)
    print(f"Frame {frame_id} processed, {len(online_targets)} vehicles tracked.")

# Release resources
cap.release()
out.release()
result_file.close()